In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
import datetime as dt
from math import acos, sin, cos, pi
%matplotlib inline

In [2]:
df1 = pd.read_csv("boston_listings_summary.csv")
df2 = pd.read_csv("mbta_metadata.csv")

Stations

In [3]:
stations = df2['station'].unique()

In [4]:
stations

array(['Alewife Station', 'Davis Station', 'Porter Square Station',
       'Harvard Square Station', 'Central Square Station',
       'Kendall Station', 'Charles/MGH Station', 'Park Street Station',
       'Downtown Crossing Station', 'South Station', 'Broadway Station',
       'Andrew Station', 'JFK/UMass Station', 'North Quincy Station',
       'Wollaston Station', 'Quincy Center Station',
       'Quincy Adams Station', 'Braintree Station', 'Savin Hill Station',
       'Fields Corner', 'Shawmut Station', 'Ashmont Station',
       'Cedar Grove Station', 'Butler Station', 'Milton Station',
       'Central Avenue Station', 'Valley Road Station',
       'Capen Street Station', 'Mattapan Station', 'Oak Grove Station',
       'Malden Center Station', 'Wellington Station', 'Assembly Station',
       'Sullivan Square Station', 'Community College Station',
       'North Station', 'Haymarket Station', 'State Station',
       'Chinatown Station', 'Tufts Medical Center Station',
       'Back Bay

In [5]:
def distance_km(lat1, long1, lat2, long2):
    d = acos(
        sin(lat1* (pi/180)) * sin(lat2* (pi/180)) + 
        cos(lat1* (pi/180)) * cos(lat2* (pi/180)) * cos(long2* (pi/180) - (long1* (pi/180)))
    ) * 6371 
    return(d)

In [6]:
def closest_station(lat, long):
    distances = {}
    for s in stations:
        station = df2.loc[df2['station'] == s]
        lat2 = station['ll__001']
        long2 = station['ll__002']
        distances[s] = distance_km(lat, long, lat2, long2)
    distance_min, station = min(distances.values()), min(distances, key=distances.get)
    return(station)

In [7]:
def closest_station_transform(row):
    return(closest_station(row['latitude'], row['longitude']))

In [10]:
%%time
df1['closest_station'] = df1.apply(lambda row: closest_station(row['latitude'], row['longitude']) ,axis=1)

CPU times: user 8min 56s, sys: 3.19 s, total: 8min 59s
Wall time: 8min 57s


In [11]:
df3 = pd.merge(df1,
              df2[['stop_id', 'station', 'line__001', 'line__002', 'll__001', 'll__002']],
              left_on = 'closest_station',
              right_on = 'station',
              how = 'left')

In [12]:
%%time

df3['closest_station_distance'] = df3.\
apply(lambda row: distance_km(row['latitude'], row['longitude'], row['ll__001'], row['ll__002']) ,axis=1)

CPU times: user 158 ms, sys: 1.93 ms, total: 160 ms
Wall time: 159 ms


In [13]:
def closest_station_km_t(row, dist):
    if row['closest_station_distance'] <= dist:
        return(row['closest_station'])
    else:
        return(None)

In [14]:
df3['closest_station_1km'] = df3.apply(lambda row: closest_station_km_t(row, 1) ,axis=1)
df3['closest_station_2km'] = df3.apply(lambda row: closest_station_km_t(row, 2) ,axis=1)
df3['closest_station_3km'] = df3.apply(lambda row: closest_station_km_t(row, 3) ,axis=1)
df3['closest_station_4km'] = df3.apply(lambda row: closest_station_km_t(row, 4) ,axis=1)
df3['closest_station_5km'] = df3.apply(lambda row: closest_station_km_t(row, 5) ,axis=1)
df3['closest_station_10km'] = df3.apply(lambda row: closest_station_km_t(row, 10) ,axis=1)

In [16]:
df3.head(20)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,line__002,ll__001,ll__002,closest_station_distance,closest_station_1km,closest_station_2km,closest_station_3km,closest_station_4km,closest_station_5km,closest_station_10km
0,3781,HARBORSIDE-Walk to subway,4804,Frank,NaN,East Boston,42.365241,-71.029361,Entire home/apt,125,...,NaN,42.369189,-71.039627,0.950772,Maverick Station,Maverick Station,Maverick Station,Maverick Station,Maverick Station,Maverick Station
1,5506,**$79 Special ** Private! Minutes to center!,8229,Terry,NaN,Roxbury,42.329809,-71.095595,Entire home/apt,145,...,NaN,42.331428,-71.095613,0.180086,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station
2,6695,$99 Special!! Home Away! Condo,8229,Terry,NaN,Roxbury,42.329941,-71.093505,Entire home/apt,169,...,NaN,42.331428,-71.095613,0.239538,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station
3,6976,Mexican Folk Art Haven in Boston Residential Area,16701,Phil,NaN,Roslindale,42.292438,-71.135765,Private room,65,...,NaN,42.300647,-71.115869,1.873793,None,Forest Hills Station,Forest Hills Station,Forest Hills Station,Forest Hills Station,Forest Hills Station
4,8789,Curved Glass Studio/1bd facing Park,26988,Anne,NaN,Downtown,42.359187,-71.062651,Entire home/apt,99,...,NaN,42.361420,-71.062115,0.252185,Bowdoin Station,Bowdoin Station,Bowdoin Station,Bowdoin Station,Bowdoin Station,Bowdoin Station
5,8792,Large 1 Bed facing State House,26988,Anne,NaN,Downtown,42.358497,-71.062011,Entire home/apt,154,...,green,42.356487,-71.062563,0.228049,Park Street Station,Park Street Station,Park Street Station,Park Street Station,Park Street Station,Park Street Station
6,9765,[1294] Grand Studio - South End,25188,Seamless,NaN,South End,42.342594,-71.079421,Entire home/apt,229,...,NaN,42.345685,-71.081749,0.393337,Prudential Station,Prudential Station,Prudential Station,Prudential Station,Prudential Station,Prudential Station
7,9824,[1273] Fab Studio- close to Newbury,25188,Seamless,NaN,Back Bay,42.349496,-71.085954,Entire home/apt,209,...,NaN,42.347939,-71.087939,0.237905,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center
8,9827,[1168-1C] Luxury 1 BR High-Rise in Beacon Hill,25188,Seamless,NaN,Downtown,42.352149,-71.063301,Entire home/apt,389,...,NaN,42.352501,-71.062610,0.069014,Chinatown Station,Chinatown Station,Chinatown Station,Chinatown Station,Chinatown Station,Chinatown Station
9,9855,[1330-1D] Lux 1BR w/ Den - Fenway,25188,Seamless,NaN,Fenway,42.343371,-71.098708,Entire home/apt,259,...,NaN,42.345463,-71.104259,0.512074,Fenway Station,Fenway Station,Fenway Station,Fenway Station,Fenway Station,Fenway Station


In [17]:
list(df3)

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'closest_station',
 'stop_id',
 'station',
 'line__001',
 'line__002',
 'll__001',
 'll__002',
 'closest_station_distance',
 'closest_station_1km',
 'closest_station_2km',
 'closest_station_3km',
 'closest_station_4km',
 'closest_station_5km',
 'closest_station_10km']

In [18]:
# data set to count the number of listings in a certain km range of each station

count_listings = df3.groupby(['closest_station_1km'])[['id']].count()\
    .reset_index().rename(columns = {"id": "nb_listings1", "closest_station_1km" : "station"})

count_listings_2 = df3.groupby(['closest_station_2km'])[['id']].count()\
    .reset_index().rename(columns = {"id": "nb_listings2", "closest_station_2km" : "station"})

count_listings_3 = df3.groupby(['closest_station_3km'])[['id']].count()\
    .reset_index().rename(columns = {"id": "nb_listings3", "closest_station_3km" : "station"})

count_listings_4 = df3.groupby(['closest_station_4km'])[['id']].count()\
    .reset_index().rename(columns = {"id": "nb_listings4", "closest_station_4km" : "station"})

count_listings_5 = df3.groupby(['closest_station_5km'])[['id']].count()\
    .reset_index().rename(columns = {"id": "nb_listings5", "closest_station_5km" : "station"})

count_listings_10 = df3.groupby(['closest_station_10km'])[['id']].count()\
    .reset_index().rename(columns = {"id": "nb_listings10", "closest_station_10km" : "station"})

In [19]:
df4 = pd.merge(df2,
              count_listings,
              on = "station", how = 'left')

df4 = pd.merge(df4,
              count_listings_2,
              on = "station", how = 'left')

df4 = pd.merge(df4,
              count_listings_3,
              on = "station", how = 'left')

df4 = pd.merge(df4,
              count_listings_4,
              on = "station", how = 'left')

df4 = pd.merge(df4,
              count_listings_5,
              on = "station", how = 'left')

df4 = pd.merge(df4,
              count_listings_10,
              on = "station", how = 'left')

In [20]:
df4

,stop_id,stop_count,stop_reviews,station,line__001,line__002,x,y,ll__001,ll__002,nb_listings1,nb_listings2,nb_listings3,nb_listings4,nb_listings5,nb_listings10
0,101,1,1,Alewife Station,red,NaN,640.0,196.0,42.395781,-71.142059,NaN,NaN,NaN,NaN,NaN,NaN
1,102,1,1,Davis Station,red,NaN,715.0,269.0,42.397096,-71.121853,NaN,NaN,NaN,NaN,NaN,NaN
2,103,1,1,Porter Square Station,red,NaN,785.0,344.0,42.388453,-71.119147,NaN,NaN,NaN,NaN,NaN,NaN
3,104,1,1,Harvard Square Station,red,NaN,862.0,420.0,42.373562,-71.118911,11.0,34.0,34.0,34.0,34.0,34.0
4,105,1,1,Central Square Station,red,NaN,938.0,495.0,42.365556,-71.103847,1.0,1.0,1.0,1.0,1.0,1.0
5,106,1,1,Kendall Station,red,NaN,1012.0,570.5,42.362575,-71.086209,NaN,NaN,NaN,NaN,NaN,NaN
6,107,1,1,Charles/MGH Station,red,NaN,1070.0,630.0,42.361212,-71.070674,208.0,208.0,208.0,208.0,208.0,208.0
7,108,1,1,Park Street Station,red,green,1165.0,720.0,42.356487,-71.062563,60.0,60.0,60.0,60.0,60.0,60.0
8,109,1,1,Downtown Crossing Station,red,orange,1250.0,810.0,42.355631,-71.060460,84.0,84.0,84.0,84.0,84.0,84.0
9,110,1,1,South Station,red,NaN,1318.0,878.0,42.352396,-71.055310,63.0,85.0,85.0,85.0,85.0,85.0


In [21]:
print(len(df4))
df4['nb_listings1'].isna().sum()

121


38

In [22]:
print(len(df4))
df4['nb_listings10'].isna().sum()

121


35

In [23]:
df4.to_csv("count_listings_stations.csv", index = False)

In [25]:
list(df3)

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365',
 'closest_station',
 'stop_id',
 'station',
 'line__001',
 'line__002',
 'll__001',
 'll__002',
 'closest_station_distance',
 'closest_station_1km',
 'closest_station_2km',
 'closest_station_3km',
 'closest_station_4km',
 'closest_station_5km',
 'closest_station_10km']

In [26]:
df3['room_type'].unique()

array(['Entire home/apt', 'Private room', 'Shared room'], dtype=object)

In [27]:
df3.to_csv("listings_summary_w_stations.csv", index = False)

In [4]:
list(df2)

['stop_id',
 'stop_count',
 'stop_reviews',
 'station',
 'line__001',
 'line__002',
 'x',
 'y',
 'll__001',
 'll__002']

---
# Testing interactivity of page

In [3]:
df = pd.read_csv("listings_summary_w_stations.csv")

In [5]:
df = pd.merge(df, 
             df2[["stop_id", "x", "y"]],
             on = "stop_id", 
             how = "left")

In [6]:
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,ll__002,closest_station_distance,closest_station_1km,closest_station_2km,closest_station_3km,closest_station_4km,closest_station_5km,closest_station_10km,x,y
0,3781,HARBORSIDE-Walk to subway,4804,Frank,NaN,East Boston,42.365241,-71.029361,Entire home/apt,125,...,-71.039627,0.950772,Maverick Station,Maverick Station,Maverick Station,Maverick Station,Maverick Station,Maverick Station,1487.0,530.0
1,5506,**$79 Special ** Private! Minutes to center!,8229,Terry,NaN,Roxbury,42.329809,-71.095595,Entire home/apt,145,...,-71.095613,0.180086,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,850.0,1210.0
2,6695,$99 Special!! Home Away! Condo,8229,Terry,NaN,Roxbury,42.329941,-71.093505,Entire home/apt,169,...,-71.095613,0.239538,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,Roxbury Crossing Station,850.0,1210.0
3,6976,Mexican Folk Art Haven in Boston Residential Area,16701,Phil,NaN,Roslindale,42.292438,-71.135765,Private room,65,...,-71.115869,1.873793,NaN,Forest Hills Station,Forest Hills Station,Forest Hills Station,Forest Hills Station,Forest Hills Station,579.0,1483.0
4,8789,Curved Glass Studio/1bd facing Park,26988,Anne,NaN,Downtown,42.359187,-71.062651,Entire home/apt,99,...,-71.062115,0.252185,Bowdoin Station,Bowdoin Station,Bowdoin Station,Bowdoin Station,Bowdoin Station,Bowdoin Station,1168.0,595.0
5,8792,Large 1 Bed facing State House,26988,Anne,NaN,Downtown,42.358497,-71.062011,Entire home/apt,154,...,-71.062563,0.228049,Park Street Station,Park Street Station,Park Street Station,Park Street Station,Park Street Station,Park Street Station,1165.0,720.0
6,9765,[1294] Grand Studio - South End,25188,Seamless,NaN,South End,42.342594,-71.079421,Entire home/apt,229,...,-71.081749,0.393337,Prudential Station,Prudential Station,Prudential Station,Prudential Station,Prudential Station,Prudential Station,830.0,870.0
7,9824,[1273] Fab Studio- close to Newbury,25188,Seamless,NaN,Back Bay,42.349496,-71.085954,Entire home/apt,209,...,-71.087939,0.237905,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center,Hynes Convention Center,797.0,812.0
8,9827,[1168-1C] Luxury 1 BR High-Rise in Beacon Hill,25188,Seamless,NaN,Downtown,42.352149,-71.063301,Entire home/apt,389,...,-71.062610,0.069014,Chinatown Station,Chinatown Station,Chinatown Station,Chinatown Station,Chinatown Station,Chinatown Station,1135.0,926.0
9,9855,[1330-1D] Lux 1BR w/ Den - Fenway,25188,Seamless,NaN,Fenway,42.343371,-71.098708,Entire home/apt,259,...,-71.104259,0.512074,Fenway Station,Fenway Station,Fenway Station,Fenway Station,Fenway Station,Fenway Station,625.0,856.0


In [7]:
df.to_csv("listings_summary_w_stations.csv", index = False)